# Амортизационный анализ

## Идея через пример

Рассмотрим классическую реализацию динамического массива. Нам для целей примера нужны только операции get и push_back

In [6]:
class SimpleDynamicArray:
    def __init__(self, capacity=1):
        self.capacity = capacity
        self.size = 0
        self.array = [None] * capacity

    def push_back(self, element):
        # Когда capacity заканчивается, создаем новый массив вдвое большей длины
        if self.size == self.capacity:
            new_capacity = self.capacity * 2
            new_array = [None] * new_capacity

            for i in range(self.size):
                new_array[i] = self.array[i]

            self.array = new_array
            self.capacity = new_capacity

        self.array[self.size] = element
        self.size += 1

    def get(self, index):
        if 0 <= index < self.size:
            return self.array[index]
        raise IndexError("Index out of range")

Заметим что, если в данных момент size < capacity, то push_back нового элемента занимает $O(1)$. Однако если size = capacity, то push_back делается уже за целых $O(n)$.

То есть, строго говоря, асимптотика функции push_back в общем - $O(n)$. Но мы же понимаем, что при многократных запросах на добавление ***сильно чаще*** триггерится именно первый вариант поведения с $O(1)$.

Амортизационный анализ позволяет формализовать это ощущение в виде понятия амортизированного времени и предоставляет инструменты подсчета его асимптотики

## Базовые понятия

Пусть имеется структура данных c набором состояний $S = \{ s_0, s_1, s_2, \dots \}$, поддерживающая семейство операций $\mathcal{O}=\{o_1,o_2,\dots\}$.

Для любой операции $o$ в состоянии $s$ определена её **фактическая цена выполнения** $T(o, s)\in\mathbb{R}_{\ge0}$. На самом деле, полезно будет отойти от RAM модели и воспринимать $T(o, s)$ просто как абстрактную стоимость, т.к. амортизационный анализ в принципе более общая идея

Рассмотрим последовательность операций $\sigma=(o_1,o_2,\dots,o_n)$, выполняемую на начальном состоянии $s_0$. Через $s_i$ обозначим состояние после выполнения первых $i$ операций. Тогда суммарная фактическая стоимость очевидно равна $T(\sigma) \;=\;\sum_{i=1}^n T\bigl(o_i, s_{i-1}\bigr).
$

***Определение:*** Амортизационная стоимость $\hat{T}(o_i)$ - условная стоимость операции $o_i$, выбранная так, чтобы просто выполнялось

$$
\hat{T}(\sigma) = \sum_{i=1}^n \hat{T}(o_i)\; \ge\;T(\sigma),
$$

для **любой** последовательности $\sigma$.

Тогда **амортизационная стоимость на операцию** не превышает
$\max_{o\in\mathcal{O}}\hat{T}(o).$

За этим лежит простая идея. Мы с помощью этого зазора просто потом будем "подтягивать" простые операции (например $O(1)$), и "занижать" сложные (например $O(n)$)

**Идея через пример:** Рассмотрим структуру данных, в которой все действия дешевые, например $O(1)$, но иногда ей нужно перестраиваться, например чтобы поддерживать свой инвариант, и действие единожды выполняется за $O(n)$. Мы хотели бы уметь отображать такое накопления "потенциальной энергии" в структуре данных и как-то грамотно это описывать

Амортизационный анализ (amortized analysis) изучает среднюю стоимость операций в последовательности, гарантируя, что «дорогие» операции компенсируются «дешёвыми» в рамках всей последовательности. В отличие от среднего (probabilistic) анализа, амортизационный анализ не требует предположений о распределении входных данных: он даёт детерминированный верхний предел на среднюю по последовательности стоимость.


## Аггрегационный метод

## Метод кредита

Вводить функцию рангов

## Метод потенциалов

## From gpt

https://chatgpt.com/share/68145fe1-b298-800d-8a6c-d9084f150b3c

https://alvmer.github.io/complexity/amortized.html

https://habr.com/ru/articles/208624/

https://neerc.ifmo.ru/wiki/index.php?title=%D0%90%D0%BC%D0%BE%D1%80%D1%82%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7#.D0.9C.D0.B5.D1.82.D0.BE.D0.B4_.D1.83.D1.81.D1.80.D0.B5.D0.B4.D0.BD.D0.B5.D0.BD.D0.B8.D1.8F

## Основные методы анализа

### 1. Аггрегированный метод

**Идея.** Подсчитать суммарную фактическую стоимость $C_{\text{act}}(\sigma)$ для произвольной последовательности длины $n$, затем разделить на $n$:

$$
\text{амортизационная стоимость} \;\le\; \frac{C_{\text{act}}(\sigma)}{n}.
$$

**Пример.** В динамическом массиве, растущем вдвое:

* Пусть $n$ операций `push`. Полное число «копирований» при расширениях есть
  $\sum_{k=0}^{\lfloor\log n\rfloor}2^k = 2^{\lfloor\log n\rfloor+1}-1 < 2n$.
* Фактических «дорогих» операций (копирований) за $n$ вставок не более $2n$.
* Итого $C_{\text{act}}(\sigma)\le n + 2n = 3n$.
* Значит, амортизационная стоимость каждой вставки $=O(1)$.

### 2. Метод счётов (accounting method)

**Идея.** Каждой операции $o$ ставим в соответствие «счёт» (charged cost) $\hat c(o)\ge c(o)$. Разницу $\hat c(o)-c(o)$ откладываем на «депозит» (credit), который впоследствии покрывает стоимость «дорогих» операций. Необходимо гарантировать, что депозит никогда не становится отрицательным.

**Формальный критерий.** Пусть на момент $i$ суммарный депозит

$$
\Delta_i = \sum_{j=1}^i \bigl(\hat c(o_j)-c(o_j)\bigr)
$$

должен удовлетворять $\Delta_i\ge0$ для всех $i=1,\dots,n$. Тогда
$\sum\hat c(o_j)\ge\sum c(o_j)$.

### 3. Потенциальный метод

**Идея.** Вводится функция потенциала $\Phi\colon\{\text{состояния}\}\to\mathbb{R}_{\ge0}$, которая отражает «накопленную работу». Для операции $o$ на состоянии $D$ определяем

$$
\hat c(o,D)  \;=\; c(o,D)\;+\;\Phi\bigl(D'\bigr)\;-\;\Phi(D),
$$

где $D'$ — состояние после выполнения $o$. Тогда для последовательности $\sigma$

$$
\sum_{i=1}^n \hat c(o_i,D_{i-1})
= \sum_{i=1}^n c(o_i,D_{i-1})
+ \Phi(D_n)-\Phi(D_0).
$$

При условии $\Phi(D_0)=0$ и $\Phi(D)\ge0$ для всех $D$ получаем
$\sum \hat c \ge \sum c$. Максимальная амортизационная стоимость —
это $\max_{o,D}\hat c(o,D)$.

---

## Теоремы и эквивалентность методов

1. **Эквивалентность агрегированного и потенциального методов.**
   Любой результат, полученный агрегированным методом, можно формализовать через некоторый потенциал.

2. **Связь с методом счётов.**
   Можно связать «депозиты» с неотрицательными приращениями потенциала:

   $$
   \hat c(o,D)-c(o,D) = \Phi(D)-\Phi(D')
   \quad\Longleftrightarrow\quad
   \text{депозит} = -\Delta\Phi.
   $$

---

## Классические примеры

### Пример 1. Динамический массив

* **Состояние**: текущий размер массива $n$ и ёмкость $m$.
* **Потенциал**: $\Phi = 2n - m$ (неотрицателен после каждой расширения).
* **Анализ**:

  * Обычное `push`: $c=1$, изменение потенциала $\Delta\Phi = 2$.
    $\hat c = 1 + 2 = 3$.
  * При расширении: фактический $c = n+1$ (копирования + вставка),
    $\Delta\Phi = (2n - 2m) - (2n - m) = m - 2m = -m$.
    Поскольку $m=n$ до расширения, $\hat c = (n+1) - n =1$.
* **Заключение**: амортизационная стоимость операции — $O(1)$.

### Пример 2. Стек с мульти-попом

* Операции: `push` — $c=1$, `pop` — $c=1$, `multipop(k)` — $c=\min(k,h)$, где $h$ — текущая высота.
* **Агрегированный метод**: за любые $n$ операций `push` и `pop` реально выполняется не более $n$ `push` и $n$ `pop`, значит суммарно $O(n)$.
  Следовательно, амортизационная стоимость каждой — $O(1)$.

### Пример 3. DSU (с Union by Rank + Path Compression)

* Показывает амортизацию $\alpha(n)$ (обратная аккумулированная логарифмическая функция).
* Формальный анализ опирается на сложные потенциалы, задаваемые рангами и глубинами, и приводит к оценке $O(\alpha(n))$ на операцию.

---

## Вывод

Амортизационный анализ обеспечивает **строгие** детерминированные оценки среднего поведения последовательности операций без предположений о распределении входных данных. Три основных метода (агрегированный, счётов и потенциальный) эквивалентны по своей вычислительной силе и позволяют выбирать наиболее удобный способ доказательства. Примеры динамического массива, стека с мульти-попом и DSU иллюстрируют мощь этого подхода в проектировании высокопроизводительных структур данных.